In [15]:
output_path = './outputs/graphs/'
results_path = 'outputs/imdb_random_label/'

In [16]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import graph_code.graphing_utils as plotting
from copy import deepcopy

In [17]:
colwidth = 4.22716535
pagewidth = 8.7598425

In [18]:
results_dict = {}

for file in [
    f for f in os.listdir(results_path) if f.endswith("json")
]:
    with open(results_path + file) as f:
        results_temp = json.load(f)
    results_dict[file] = results_temp


In [19]:
results_idpa = {
    ds : {
        ctype : {
            run: {
                'idpa': results_dict['results_idpa.json'][ds][ctype][run]
            } for run in results_dict['results_idpa.json'][ds][ctype]
        } for ctype in results_dict['results_idpa.json'][ds]
    } for ds in results_dict['results_idpa.json']
}

results_cot = {
    ds : {
        ctype : {
            run: {
                'cot': results_dict['results_cot.json'][ds][ctype][run]
            } for run in results_dict['results_cot.json'][ds][ctype]
        } for ctype in results_dict['results_cot.json'][ds]
    } for ds in results_dict['results_cot.json']
}

results_cdr = {
    ds : {
        ctype : {
            run: {
                'cdr': results_dict['results_cdr.json'][ds][ctype][run]
            } for run in results_dict['results_cdr.json'][ds][ctype]
        } for ctype in results_dict['results_cdr.json'][ds]
    } for ds in results_dict['results_cdr.json']
}

results_nls = {
    ds : {
        ctype : {
            run: {
                'nls': results_dict['results_nls.json'][ds][ctype][run]
            } for run in results_dict['results_nls.json'][ds][ctype]
        } for ctype in results_dict['results_nls.json'][ds]
    } for ds in results_dict['results_nls.json']
}

results_dict['results_idpa.json'] = results_idpa
results_dict['results_cot.json'] = results_cot
results_dict['results_cdr.json'] = results_cdr
results_dict['results_nls.json'] = results_nls

In [20]:
def combine_results_dict(results_dict_in, results_out):
    
    # results dict in should be of form: {dataset: {corruption_type: {run: {method: {epoch: metrics}}}}}

    for dataset in results_dict_in:
        if dataset not in results_out:
            results_out[dataset] = {}

        for corruption_type in results_dict_in[dataset]:
            if corruption_type not in results_out[dataset]:
                results_out[dataset][corruption_type] = {}

            for run in results_dict_in[dataset][corruption_type]:
                if run not in results_out[dataset][corruption_type]:
                    results_out[dataset][corruption_type][run] = {}

                for method in results_dict_in[dataset][corruption_type][run]:

                    results_out[dataset][corruption_type][run][method] = [
                        dict(epoch=int(epoch), **metrics) 
                        for epoch, metrics in results_dict_in[
                            dataset
                        ][corruption_type][run][method].items() 
                        if epoch != 'corrupt_sources'
                    ]

In [21]:
combined_results = {}

for result in results_dict:
    combine_results_dict(results_dict[result], combined_results)

In [22]:
combined_results_df = []

for dataset in combined_results:
    for corruption_type in combined_results[dataset]:
        for run in combined_results[dataset][corruption_type]:
            for method in combined_results[dataset][corruption_type][run]:
                combined_results_df.append(
                    pd.json_normalize(
                        combined_results[dataset][corruption_type][run][method]
                    )
                    .assign(
                        dataset=dataset,
                        corruption_type=corruption_type,
                        run=run,
                        method=method
                    )
                    .assign(epoch=lambda x: x['epoch']+1)
                )

combined_results_df = pd.concat(combined_results_df)
combined_results_df = (
    combined_results_df
    .replace({"method": {"true": True, "false": False}})
    .astype(
        {
            "run": "int64",
        }
    )
)

In [23]:
combined_results_df = combined_results_df.melt(
    id_vars=['dataset', 'corruption_type', 'run', 'method', 'epoch'],
    var_name='metric',
    value_name='value'
)

In [24]:
(
    combined_results_df
    .loc[lambda df: df.metric == 'test_top1acc']
    .groupby(["method", 'corruption_type', "run"])
    [['value']]
    .max()
    .unstack(0)
)

value                                             
method                 False     True      cdr      cot     idpa      nls
corruption_type run                                                      
original        1    0.81528  0.81892  0.82188  0.85656  0.83604  0.84304
                2    0.82716  0.83252  0.80884  0.85984  0.83076  0.83864
                3    0.83332  0.83248  0.84464  0.84316  0.82356  0.85180
                4    0.82936  0.84192  0.83540  0.84232  0.83804  0.82844
                5    0.83556  0.83396  0.81364  0.85400  0.83356  0.83268
random_label    1    0.63964  0.68160  0.66200  0.66232  0.63620  0.68036
                2    0.65568  0.71568  0.63660  0.67560  0.65004  0.65688
                3    0.65348  0.75684  0.64776  0.68072  0.65072  0.66708
                4    0.64804  0.70436  0.64688  0.67060  0.64116  0.66188
                5    0.65348  0.73912  0.64660  0.66988  0.65256  0.65056
random_permute  1    0.84624  0.84084  0.82944  0.85340  0.83392  0.83428
                2    0.83816  0.84376  0.84124  0.85656  0.82444  0.84120
                3    0.82136  0.81592  0.83108  0.85680  0.82480  0.85160
                4    0.83232  0.83140  0.82136  0.85704  0.84108  0.83092
                5    0.83488  0.83132  0.82236  0.85624  0.83952  0.84848

In [25]:
def bold_max_value(x, model_names):
    x = x.copy()
    len_cols = x.shape[0]
    n_models = len(model_names)
    idx_bold = (
        x
        [-n_models:]
        .str.replace(" ", "")
        .str.split("±")
        .str[0]
        .astype(float)
        .argmax()
    )
    max_val = x.iloc[idx_bold+len_cols-n_models]
    max_val, max_std = map(float, max_val.split("±"))

    for i in range(len_cols-n_models, len_cols):
        val = x.iloc[i]
        val, std = map(float, val.split("±"))
        if val >= max_val - max_std and val <= max_val + max_std:
            x.iloc[i] = '\\textbf{' + x.iloc[i] + '}'
    
    return x


corruption_types = {
    "original": "Original Data",
    "random_label": "Random Label",
}

corruption_order = [
    "original",
    "random_label",
]


model_order = [
    'Co-teaching',
    'IDPA',
    'CDR',
    'Label Smoothing'
    'Standard',
    'LAP (Ours)',
]

combined_results_final_df = (
    combined_results_df
    .loc[lambda df: df.metric == 'test_top1acc']
    .groupby(["method", 'corruption_type', "run"])
    [['value']]
    .max()
    .assign(value = lambda x: x['value']*100)
    .reset_index()
    .groupby(['corruption_type', "method"])
    ['value']
    .agg(['mean', 'std'])
    .assign(
        mean_std = lambda x: 
            np.round(x['mean'], 2).astype(str) 
            + " ± "
            + np.round(x['std'], 2).astype(str),
    )
    ['mean_std']
    .to_frame()
    .reset_index()
    .rename(columns={
        "method": "LAP", 
        'corruption_type': "Noise Type", 
        "mean_std": "Top-1 Accuracy"
    })
    .replace(
        {
            "LAP": {
                False: "Standard",
                True: "LAP (Ours)",
                'cot': 'Co-teaching',
                'idpa': 'IDPA',
                'cdr': 'CDR',
                'nls': 'Label Smoothing'
            }
        }
    )
    .pivot(
        index="Noise Type",
        columns="LAP",
        values="Top-1 Accuracy"
    )
    .reset_index()
    [[
        'Noise Type',  
        'Standard', 
        'IDPA', 
        'Co-teaching', 
        'CDR', 
        'Label Smoothing', 
        'LAP (Ours)',  
    ]]
    .loc[
        lambda x: x['Noise Type'].isin(corruption_order)
    ]
    .sort_values(
        "Noise Type", key=lambda x: x.map(corruption_order.index)
    )
    .replace(
        {
            "Noise Type": corruption_types
        }
    )
    # makes bold with latex:
    .apply(
        bold_max_value,
        model_names = model_order,
        axis=1
    )
)

combined_results_final_df

LAP,Noise Type,Standard,IDPA,Co-teaching,CDR,Label Smoothing,LAP (Ours)
0,Original Data,82.81 ± 0.79,83.24 ± 0.56,\textbf{85.12 ± 0.8},82.49 ± 1.49,83.89 ± 0.91,83.2 ± 0.83
1,Random Label,65.01 ± 0.65,64.61 ± 0.71,67.18 ± 0.69,64.8 ± 0.91,66.34 ± 1.13,\textbf{71.95 ± 2.94}


In [26]:
print(
    combined_results_final_df.to_latex(index=False)
)

\begin{tabular}{lllllll}
\toprule
Noise Type & Standard & IDPA & Co-teaching & CDR & Label Smoothing & LAP (Ours) \\
\midrule
Original Data & 82.81 ± 0.79 & 83.24 ± 0.56 & \textbf{85.12 ± 0.8} & 82.49 ± 1.49 & 83.89 ± 0.91 & 83.2 ± 0.83 \\
Random Label & 65.01 ± 0.65 & 64.61 ± 0.71 & 67.18 ± 0.69 & 64.8 ± 0.91 & 66.34 ± 1.13 & \textbf{71.95 ± 2.94} \\
\bottomrule
\end{tabular}

